# Game events analysis 
This notebook is meant to analyse an average game in the data set and find correlations between events

In [ ]:
%reload_ext autoreload
%autoreload 2
import os 
import sys
import pandas as pd
import numpy as np
from datetime import datetime
import plotly 
import plotly.graph_objects as go
from datetime import timedelta

sys.path.insert(0, "./../../src/")
from data_loader import load_data

In [ ]:
%%time

seasons = load_data(single_df=True, force_recompute=False)
seasons["season_name"] = seasons["season_name"].str.replace("-", "-20")
display(seasons)

In [ ]:
# group by seasons and events
grouped_event_df = seasons.groupby(["season_name", "EVENTMSGTYPE"]).size()
# print("Sum", grouped_event_df.sum(level=0))

# normalize by number of events in a season -> then divide by all events in the season
norm_grouped_event_df = grouped_event_df.div(grouped_event_df.sum(level=0), level=0)

grouped_event_df = grouped_event_df.reset_index().round(3)
norm_grouped_event_df = norm_grouped_event_df.reset_index().round(3)
display(grouped_event_df, norm_grouped_event_df)

## Percentage of distributed events over the seasons chart

In [ ]:
trace_arr = []
unique_seasons = grouped_event_df["season_name"].unique()
unique_events = grouped_event_df["EVENTMSGTYPE"].unique()
# print(unique_events, unique_seasons)

for event_i in unique_events:
    event_df = grouped_event_df[grouped_event_df["EVENTMSGTYPE"] == event_i]
    # print(event_df.columns, event_df)
    new_trace = go.Bar(x=event_df["season_name"], y=event_df[0], name=event_i)
    trace_arr.append(new_trace)
    
fig = go.Figure(data=trace_arr)
fig.update_layout(barmode='stack', xaxis={'categoryorder':'array'})
fig.update_layout(
    title_text="Event count through the seasons", 
    xaxis_title="Season",
    yaxis_title="Count"
)
fig.show()

In [ ]:
# normalized version
trace_arr_n = []
for event_i in unique_events:
    event_df = norm_grouped_event_df[norm_grouped_event_df["EVENTMSGTYPE"] == event_i]
    new_trace = go.Bar(x=event_df["season_name"], y=event_df[0], name=event_i)
    trace_arr_n.append(new_trace)
    
fig = go.Figure(data=trace_arr_n)
fig.update_layout(barmode='stack', xaxis={'categoryorder':'array'})
fig.update_layout(
    title_text="Event count normalized through the seasons", 
    xaxis_title="Season",
    yaxis_title="Count"
)
fig.show()